### 1. Import thư viện

In [2]:
import cv2
import os
import glob
import random
import numpy as np
from random import shuffle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten

### 2.Trích xuất và xử lý khuôn mặt từ video

##### Hàm extract_frames_from_video
Hàm này trích xuất các khung hình từ video, phát hiện khuôn mặt bằng haarcascade_frontalface_default.xml, và lưu ảnh khuôn mặt đã cắt vào thư mục đầu ra.

In [3]:
def extract_frames_from_video(video_path, output_dir, num_frames=200):
    # Tạo thư mục lưu khung hình nếu chưa tồn tại
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Load bộ phân loại khuôn mặt từ file XML
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    # Hàm cắt khuôn mặt từ ảnh
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face
    
    vidcap = cv2.VideoCapture(video_path)
    if not vidcap.isOpened():
        print(f"Không thể mở video: {video_path}")
        return
    
    # Đếm tổng số khung hình trong video
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        print(f"Video {video_path} không có khung hình nào.")
        vidcap.release()
        return
    
    # Tính bước nhảy (step) để lấy đều num_frames khung hình
    step = max(1, total_frames // num_frames)  # Đảm bảo step ít nhất là 1
    frame_indices = [i * step for i in range(num_frames) if i * step < total_frames]
    
    # Tạo các chỉ mục ngẫu nhiên
    random_indices = random.sample(range(200), len(frame_indices))
    
    # Trích xuất, cắt khuôn mặt và lưu
    count = 0
    for idx, random_idx in zip(frame_indices, random_indices):
        # Đặt con trỏ video đến khung hình cần lấy
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        success, image = vidcap.read()
        if not success:
            print(f"Không thể đọc khung hình {idx} từ video {video_path}")
            continue
        
        # Xoay ảnh nếu cần
        image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
        # Cắt khuôn mặt
        cropped_face = face_cropped(image)
        if cropped_face is not None:
            # Resize khuôn mặt về kích thước cố định
            face = cv2.resize(cropped_face, (200, 200))
            # Chuyển sang grayscale 
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            # Lưu khuôn mặt với chỉ mục ngẫu nhiên
            frame_path = os.path.join(output_dir, f"face_{random_idx}.jpg")
            cv2.imwrite(frame_path, face)
            count += 1
    
    # Giải phóng video
    vidcap.release()
    print(f"Đã trích xuất và lưu {count} khuôn mặt từ video {video_path} vào {output_dir}")



##### Hàm process_videos_in_directory
Hàm này duyệt qua tất cả file video trong thư mục video_dir, gọi extract_frames_from_video để xử lý từng video và lưu kết quả vào các thư mục con trong training_data_dir.

In [4]:
def process_videos_in_directory(video_dir, training_data_dir, num_frames=1):
    # Tạo thư mục training_data nếu chưa tồn tại
    if not os.path.exists(training_data_dir):
        os.makedirs(training_data_dir)
    
    # Duyệt qua tất cả các file video trong thư mục video_dir
    video_paths = glob.glob(os.path.join(video_dir, "*.mp4")) 
    for video_path in video_paths:
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        output_dir = os.path.join(training_data_dir, video_name)
        
        # Trích xuất khung hình, cắt khuôn mặt và lưu vào thư mục con
        extract_frames_from_video(video_path, output_dir, num_frames)



In [5]:
video_dir = "video"  # Thư mục chứa các video
training_data_dir = "training_data"  # Thư mục lưu dữ liệu huấn luyện
process_videos_in_directory(video_dir, training_data_dir, num_frames=200)

Đã trích xuất và lưu 145 khuôn mặt từ video video\ctt.mp4 vào training_data\ctt
Đã trích xuất và lưu 135 khuôn mặt từ video video\hoang.mp4 vào training_data\hoang
Đã trích xuất và lưu 123 khuôn mặt từ video video\kmt.mp4 vào training_data\kmt


### 3. Tạo nhãn
Gán nhãn (label) dưới dạng vector one-hot cho ảnh dựa trên tên thư mục chứa ảnh.

In [6]:
def my_label(image_name):
    name = os.path.basename(os.path.dirname(image_name))
    labels = {
        "ctt": np.array([1, 0, 0]),
        "hoang": np.array([0, 1, 0]),
        "kmt": np.array([0, 0, 1])
    }
    return labels.get(name, None)

### 4. Tạo data
Chuẩn bị dữ liệu huấn luyện và kiểm tra

In [7]:
def my_data(base_dir="training_data"):
    data = []
    for person_dir in tqdm(os.listdir(base_dir)):
        person_path = os.path.join(base_dir, person_dir)
        if os.path.isdir(person_path):
            for img in os.listdir(person_path):
                path = os.path.join(person_path, img)
                img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                if img_data is None:
                    continue
                img_data = cv2.resize(img_data, (50, 50))
                data.append([np.array(img_data), my_label(path)])
    shuffle(data)  # Xáo trộn dữ liệu
    return data



Chia dữ liệu thành tập train/test, và định dạng dữ liệu cho mô hình CNN

In [8]:
# Tạo dữ liệu
data = my_data()

# Chia tập train và test bằng train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)

X_train = np.array([i[0] for i in train]).reshape(-1, 50, 50, 1)
print("X_train shape:", X_train.shape)
y_train = np.array([i[1] for i in train])  # Chuyển thành numpy array

X_test = np.array([i[0] for i in test]).reshape(-1, 50, 50, 1)
print("X_test shape:", X_test.shape)
y_test = np.array([i[1] for i in test])  # Chuyển thành numpy array

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:06<00:00,  2.28s/it]

X_train shape: (428, 50, 50, 1)
X_test shape: (107, 50, 50, 1)


### 4. Định nghĩa và huấn luyện mô hình CNN
Định nghĩa một mô hình Convolutional Neural Network (CNN) bằng Keras, biên dịch mô hình, và huấn luyện nó trên dữ liệu X_train, y_train, với tập kiểm tra X_test, y_test.

In [9]:
model = Sequential()

# Lớp tích chập và pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 1), padding='same'))
model.add(MaxPooling2D((2, 2)))  # Giảm từ 50x50 -> 25x25

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))  # Giảm từ 25x25 -> 12x12

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))  # Giảm từ 12x12 -> 6x6

# Làm phẳng và fully connected
model.add(Flatten())  # 6x6x128 = 4608
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # 3 lớp đầu ra

# Biên dịch mô hình
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# In tóm tắt mô hình
model.summary()

# Huấn luyện mô hình
model.fit(X_train, y_train, 
          epochs=12, 
          validation_data=(X_test, y_test), 
          verbose=1)

d:\anacoda\envs\python_3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 50, 50, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 25, 25, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 25, 25, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,454,019 (9.36 MB)

 Trainable params: 2,454,019 (9.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/12
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 166ms/step - accuracy: 0.3911 - loss: 50.2932 - val_accuracy: 0.8131 - val_loss: 0.4188
Epoch 2/12
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.8534 - loss: 0.4123 - val_accuracy: 1.0000 - val_loss: 0.0498
Epoch 3/12
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.9609 - loss: 0.0871 - val_accuracy: 0.9907 - val_loss: 0.0205
Epoch 4/12
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.9909 - loss: 0.0272 - val_accuracy: 0.9907 - val_loss: 0.0085
Epoch 5/12
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 0.9948 - loss: 0.0202 - val_accuracy: 1.0000 - val_loss: 0.0024
Epoch 6/12
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.9962 - loss: 0.0126 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 7/12
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 1.0000 - val_loss: 8.2403e-04
Epoch 8/12
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 1.0000 - loss: 9.2848e-04 - val_accuracy

### 5. Trực quan dữ liệu và dự đoán

Chuẩn bị dữ liệu để dự đoán

In [10]:
def data_for_visualization(base_dir="training_data"):
    Vdata = []
    for person_dir in tqdm(os.listdir(base_dir)):
        person_path = os.path.join(base_dir, person_dir)
        if os.path.isdir(person_path):
            for img in os.listdir(person_path):
                path = os.path.join(person_path, img)
                img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                if img_data is None:
                    continue
                img_data = cv2.resize(img_data, (50, 50))
                label = my_label(path)
                if label is not None:
                    Vdata.append([np.array(img_data), label])
    random.shuffle(Vdata)
    return Vdata

#### Dự đoán và hiển thị kết quả trên dữ liệu Visualization
Chuẩn bị dữ liệu ảnh (X_vis) và nhãn (y_vis), sau đó dùng mô hình đã huấn luyện để dự đoán và hiển thị kết quả so sánh giữa nhãn dự đoán và nhãn thực tế.

In [11]:
Vdata = data_for_visualization()
X_vis = np.array([i[0] for i in Vdata]).reshape(-1, 50, 50, 1)
print("X_vis shape:", X_vis.shape)
y_vis = np.array([i[1] for i in Vdata])
predictions = model.predict(X_vis)
for i, pred in enumerate(predictions):
    label_idx = np.argmax(pred)
    label_name = ['ctt', 'hoang', 'kmt'][label_idx]
    true_label_idx = np.argmax(y_vis[i])
    true_label_name = ['ctt', 'hoang', 'kmt'][true_label_idx]
    print(f"Ảnh {i+1}: Dự đoán = {label_name} ({pred[label_idx]:.4f}), Thực tế = {true_label_name}")

100%|██████████| 3/3 [00:00<00:00,  7.92it/s]


X_vis shape: (535, 50, 50, 1)
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step
Ảnh 1: Dự đoán = hoang (1.0000), Thực tế = hoang
Ảnh 2: Dự đoán = kmt (1.0000), Thực tế = kmt
Ảnh 3: Dự đoán = hoang (1.0000), Thực tế = hoang
Ảnh 4: Dự đoán = kmt (1.0000), Thực tế = kmt
Ảnh 5: Dự đoán = ctt (1.0000), Thực tế = ctt
Ảnh 6: Dự đoán = ctt (1.0000), Thực tế = ctt
Ảnh 7: Dự đoán = ctt (1.0000), Thực tế = ctt
Ảnh 8: Dự đoán = ctt (1.0000), Thực tế = ctt
Ảnh 9: Dự đoán = ctt (1.0000), Thực tế = ctt
Ảnh 10: Dự đoán = kmt (1.0000), Thực tế = kmt
Ảnh 11: Dự đoán = ctt (1.0000), Thực tế = ctt
Ảnh 12: Dự đoán = kmt (1.0000), Thực tế = kmt
Ảnh 13: Dự đoán = kmt (1.0000), Thực tế = kmt
Ảnh 14: Dự đoán = kmt (1.0000), Thực tế = kmt
Ảnh 15: Dự đoán = ctt (1.0000), Thực tế = ctt
Ảnh 16: Dự đoán = kmt (1.0000), Thực tế = kmt
Ảnh 17: Dự đoán = hoang (1.0000), Thực tế = hoang
Ảnh 18: Dự đoán = ctt (1.0000), Thực tế = ctt
Ảnh 19: Dự đoán = hoang (1.0000), Thực tế = hoang
Ảnh 20: Dự đoán = kmt (1.0000), Thực tế = kmt
Ảnh

## 6. Kết luận
Dự án này dùng để đánh giá một hệ thống nhận diện khuôn mặt dựa trên mạng nơ-ron tích chập (CNN) sử dụng Python và các thư viện như OpenCV, NumPy, và TensorFlow/Keras.